# Vocab and Encoder
### Collecting our Vocabulary and training the encoder for the first time onlyyyy (it is done and you already have the encoder)

# 1) BB as a sequence (choose a cell from them)

In [2]:
# RUN this cell if you consider the BB as a sentence

'''
import LLNormalizer
import os
import json
from tqdm import tqdm
import re
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
from imblearn.over_sampling import SMOTE


vocabulary = set(['<pad>'])
max_size = 0


X_train = []
Y_train = []

for jsonfile in tqdm(os.listdir('/kaggle/input/dataaa')):
    if(not jsonfile.endswith('.json')):
      continue
    with open ('/kaggle/input/dataaa/'+jsonfile, 'r') as f:
        jsondict = json.load(f)
    for k,v in jsondict.items():
        bbMerged = '\n'.join(v[:-1])
        basicBlockContents = bbMerged
        basicBlockClass = re.findall('\d+', jsonfile)[0] #take the first digit in the filename as the CVE class
        tokenizedBB = LLNormalizer.Normalize_Tokenize_LLVM(basicBlockContents)
        max_size = max(max_size, len(tokenizedBB))
        vocabulary.update(tokenizedBB)
        X_train.append(tokenizedBB)
        Y_train.append(basicBlockClass)

print(f"our Vocab Size = {len(vocabulary)}")
print(f"Size of data = {len(X_train)}")
print(f"max size is {max_size}")

encoder = LabelEncoder()
encoder.fit(list(vocabulary))
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder,f)
    
    
#Encoding each class
ClassLabelEncoder = LabelEncoder().fit(Y_train)
Y_train = ClassLabelEncoder.transform(Y_train)

#Dont forget to use this encoder during inference
with open('ClassLabelEncoder.pkl', 'wb') as f:
    pickle.dump(ClassLabelEncoder,f)

numClasses = len(set(Y_train))
print(f"We have {numClasses} Classes")
'''


'\nimport LLNormalizer\nimport os\nimport json\nfrom tqdm import tqdm\nimport re\nimport pickle\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.model_selection import train_test_split\nimport numpy as np\nimport multiprocessing\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\nfrom imblearn.over_sampling import SMOTE\n\n\nvocabulary = set([\'<pad>\'])\nmax_size = 0\n\n\nX_train = []\nY_train = []\n\nfor jsonfile in tqdm(os.listdir(\'/kaggle/input/dataaa\')):\n    if(not jsonfile.endswith(\'.json\')):\n      continue\n    with open (\'/kaggle/input/dataaa/\'+jsonfile, \'r\') as f:\n        jsondict = json.load(f)\n    for k,v in jsondict.items():\n        bbMerged = \'\n\'.join(v[:-1])\n        basicBlockContents = bbMerged\n        basicBlockClass = re.findall(\'\\d+\', jsonfile)[0] #take the first digit in the filename as the CVE class\n        tokenizedBB = LLNormalizer.Normalize_Tokenize_LLVM(basicBlockContents)\n        max_size = max(max_size, len(to

# 2) Function as a sentence

In [3]:
# RUN this cell if you consider the BB as a sentence

import LLNormalizer
import os
import json
from tqdm import tqdm
import re
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
from imblearn.over_sampling import SMOTE
import CodePreprocessing


vocabulary = set(['<pad>'])
max_size = 0


X_train = []
Y_train = []


#I assume the dataset is inside a folder called llfiles
allLLfiles = os.listdir('llfiles')
for llfile in tqdm(allLLfiles):
    if(not llfile.endswith('.ll')):
        continue
   
    fn_head, fn_body = CodePreprocessing.functions_preprocessing('llfiles/'+llfile, '',writeJson=False)
    for i in range(len(fn_head)):
        classification = re.findall('\{\s+CVE(\d+)', fn_head[i])
        if(not classification):
            classification = '0'
        tokenizedFN = LLNormalizer.Normalize_Tokenize_LLVM(fn_body[i])
        max_size = max(max_size, len(tokenizedFN))
        vocabulary.update(tokenizedFN)
        X_train.append(tokenizedFN)
        Y_train.append(classification[0])

print(f"our Vocab Size = {len(vocabulary)}")
print(f"Size of data = {len(X_train)}")
print(f"max size is {max_size}")

encoder = LabelEncoder()
encoder.fit(list(vocabulary))
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder,f)
    
    
#Encoding each class
ClassLabelEncoder = LabelEncoder().fit(Y_train)
Y_train = ClassLabelEncoder.transform(Y_train)

#Dont forget to use this encoder during inference
with open('ClassLabelEncoder.pkl', 'wb') as f:
    pickle.dump(ClassLabelEncoder,f)

numClasses = len(set(Y_train))
print(f"We have {numClasses} Classes")

100%|██████████| 60/60 [01:54<00:00,  1.90s/it]

our Vocab Size = 4947
Size of data = 29444
max size is 53619
We have 2 Classes


# Data Loader
### We Load our dataset file by file from here, and each file is encoded before being stored in our X_train
### also we encode our classes from 190, 191 to 1 and 2 and so on

### We have 2 loaders, one for BBs and the other for Functions
# Choose only 1 function from them to train

In [4]:
def BasicBlockDataLoader(folderPath, lastfileIndex, filesPerBatch=6):
    #Load the encoder for each token
    with open('encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)
    padderLabel = encoder.transform(['<pad>'])[0]
    print(f"padderLabel is {padderLabel}")


    X_train = []
    Y_train = []
    cnt = lastfileIndex
    allJsons = os.listdir(folderPath)
    for jsonfile in tqdm(allJsons[lastfileIndex: lastfileIndex + filesPerBatch]):
        if(not jsonfile.endswith('.json') or (cnt-lastfileIndex > filesPerBatch)):
          continue
        cnt += 1

        with open (folderPath+'/'+jsonfile, 'r', encoding='utf-8') as f:
            jsondict = json.load(f)

        for k,v in jsondict.items():
            bbMerged = '\n'.join(v[:-1])
            basicBlockContents = bbMerged
            basicBlockClass = re.findall('\d+', jsonfile)[0] #take the first digit in the filename as the CVE class
            tokenizedBB = LLNormalizer.Normalize_Tokenize_LLVM(basicBlockContents)
            tokenizedBB = np.array(tokenizedBB)
            tokenizedBB = encoder.transform(tokenizedBB)
            X_train.append(tokenizedBB)
            Y_train.append(int(basicBlockClass))
    #Encoding each class
    #Load the Class encoder for each label
    with open('ClassLabelEncoder.pkl', 'rb') as f:
        ClassLabelEncoder = pickle.load(f)
    Y_train = ClassLabelEncoder.transform(Y_train)
    return X_train, Y_train, padderLabel


# same function but loads dataset function by function, not BB by BB
def FunctionDataLoader(folderPath, lastfileIndex, filesPerBatch=6):
    #Load the encoder for each token
    with open('encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)
    padderLabel = encoder.transform(['<pad>'])[0]
   # print(f"padderLabel is {padderLabel}")

    X_train = []
    Y_train = []

    cnt = lastfileIndex
    allLLfiles = os.listdir(folderPath)
    for llfile in tqdm(allLLfiles):
        if(not llfile.endswith('.ll') or (cnt-lastfileIndex > filesPerBatch) ):
            continue
        cnt+=1
        fn_head, fn_body = CodePreprocessing.functions_preprocessing(folderPath+'/'+llfile, '',writeJson=False)
        for i in range(len(fn_head)):
            classification = re.findall('\{\s+CVE(\d+)', fn_head[i])
            if(not classification):
                classification = '0'
            tokenizedFN = LLNormalizer.Normalize_Tokenize_LLVM(fn_body[i])
            
            tokenizedFN = np.array(tokenizedFN)
            tokenizedFN = encoder.transform(tokenizedFN)
            X_train.append(tokenizedFN)
            Y_train.append(classification[0])

    
    #Encoding each class
    #Load the Class encoder for each label
    with open('ClassLabelEncoder.pkl', 'rb') as f:
        ClassLabelEncoder = pickle.load(f)
    Y_train = ClassLabelEncoder.transform(Y_train)
    return X_train, Y_train, padderLabel

In [5]:
#print(X_train[0])
#print(Y_train[0:5])

# Padding & SMOTE

### Padding and Preparing Tensors

In [6]:
def Pad_And_SMOTE(X_train, Y_train, max_size,padLabel, smoteflag):
    #Padding
    new_Train = np.zeros((len(X_train), max_size), dtype='int')
    for i,train in tqdm(enumerate(X_train)):
        new_Train[i] = np.pad(train, (0, max_size-len(train)), 'constant', constant_values=padLabel)


    #SMOTE
    if(not smoteflag):
        return new_Train, Y_train
    else:
        preserved_shape = new_Train.shape
        #print(f"the original shape is {preserved_shape}")
        # Reshape X_train to a 2D array of shape (n_samples, n_features)
        new_Train = np.reshape(new_Train, (new_Train.shape[0], -1))

        # Reshape Y_train to a 1D array of shape (n_samples,)
        Y_train = np.reshape(Y_train, (Y_train.shape[0],))

        # Apply SMOTE to oversample the minority class
        oversample = SMOTE()
        X_train_resampled, Y_train_resampled = oversample.fit_resample(new_Train, Y_train)

        # Reshape X_train_resampled to a 3D tensor of shape (num_samples, sequence_length, input_size)
        num_samples = X_train_resampled.shape[0]
        sequence_length = max_size
        input_size = 1
        X_train_resampled = X_train_resampled.reshape(num_samples, sequence_length)

        # Reshape Y_train_resampled back to a 1D tensor of shape (num_samples,)
        Y_train_resampled = np.reshape(Y_train_resampled, (Y_train_resampled.shape[0],))
        #print(X_train_resampled.shape)
        return X_train_resampled, Y_train_resampled

# Same Code but Optimized

### Loads the previous model (if present), continues training

In [7]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        logits = self.fc(hidden)
        sentence_embedding = hidden.squeeze(0)  # Extract the final hidden state as the sentence embedding
        return logits, sentence_embedding



def TrainModel(vocabulary, numClasses, new_X_tensor, Y_tensor, save_model_flag=True):
    
    # Define hyperparameters
    vocab_size = len(vocabulary)
    embedding_dim = 500
    hidden_dim = 280
    output_dim = numClasses # Number of classes
    num_layers = 3
    dropout = 0.5
    num_epochs = 10
    batch_size = 5
    learning_rate = 0.001

    # Load and preprocess your data
    # ...

    # Split your data into training, validation, and test sets
    xtrain, xval, ytrain, yval = train_test_split(new_X_tensor, Y_tensor, shuffle=True,test_size=0.3)

    train_dataset = TensorDataset(torch.LongTensor(xtrain), torch.LongTensor(ytrain))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = TensorDataset(torch.LongTensor(xval), torch.LongTensor(yval))
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    del(new_X_tensor)
    del(Y_tensor)
    del(xtrain)
    del(ytrain)

    # Initialize your model and optimizer
    if(os.path.exists('LSTM.pkl')):
        with open('LSTM.pkl', 'rb') as f:
            model = pickle.load(f).to(device='cuda')
    else:
        model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device="cuda")
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Define the learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

    # for EARLY STOPPING
    endTraining=False
    # Train your model
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        for batch, (batch_text, batch_labels) in enumerate(train_loader):
            batch_text = batch_text.to(device="cuda")
            batch_labels = batch_labels.to(device="cuda")

            optimizer.zero_grad()
            output, _ = model(batch_text)
            loss = criterion(output, batch_labels)
            loss.backward()

            # Clip the norm of the gradients to prevent explosion
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            # Compute the number of correct predictions and the total number of samples processed so far
            _, predicted = torch.max(output, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()


            # Compute the accuracy for the batch
            batch_accuracy = 100 * train_correct / train_total
            # Print the accuracy for the batch
            print(f'Epoch {epoch+1}, Batch {batch+1}/{len(train_loader)}, Batch Accuracy: {batch_accuracy:.2f}%, Loss: {loss.item():.4f}')
            optimizer.step()

            train_loss += loss.item()
            #EARLY STOPPING
            #if(loss.item() < 0.4):
            #    endTraining=True
            #    break
        #if(endTraining):
        #    break
        
        
        # Compute the average training loss for the epoch
        train_loss /= len(train_loader)

        # Evaluate your model on the validation set
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch_text, batch_labels in val_loader:
                batch_text = batch_text.to(device="cuda")
                batch_labels = batch_labels.to(device="cuda")

                output, _ = model(batch_text)
                loss = criterion(output, batch_labels)
                val_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()

        # Compute the average validation loss and accuracy for the epoch
        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        # Update the learning rate using the scheduler
        scheduler.step(val_loss)

        # Print the training and validation losses and accuracy for the epoch
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

        
    #Save the model
    if(save_model_flag):
        with open('LSTM.pkl', 'wb') as f:
            pickle.dump(model,f)
    
    return model

# Wrapping it up
### remember to choose the BB/function accordingly

In [ ]:
fileBatchSize = 7

for i in range (0,50,fileBatchSize):
    X_train, Y_train, padderLabel = FunctionDataLoader('llfiles', lastfileIndex=i, filesPerBatch= fileBatchSize)
    new_X, new_Y = Pad_And_SMOTE(X_train, Y_train, max_size, padderLabel, smoteflag=False)
    torch.cuda.empty_cache()
    new_X_tensor = torch.LongTensor(new_X)
    new_X_tensor.to(device = 'cuda')
    Y_tensor = torch.LongTensor(new_Y)
    Y_tensor.to(device='cuda')

    model = TrainModel(vocabulary, numClasses, new_X_tensor, Y_tensor, save_model_flag=True)

100%|██████████| 60/60 [00:46<00:00,  1.30it/s]
3885it [00:01, 3004.29it/s]


Epoch 1, Batch 1/544, Batch Accuracy: 0.00%, Loss: 0.7154
Epoch 1, Batch 2/544, Batch Accuracy: 50.00%, Loss: 0.5112
Epoch 1, Batch 3/544, Batch Accuracy: 66.67%, Loss: 0.2635
Epoch 1, Batch 4/544, Batch Accuracy: 75.00%, Loss: 0.0716
Epoch 1, Batch 5/544, Batch Accuracy: 80.00%, Loss: 0.0179
Epoch 1, Batch 6/544, Batch Accuracy: 83.33%, Loss: 0.0024
Epoch 1, Batch 7/544, Batch Accuracy: 85.71%, Loss: 0.0014
Epoch 1, Batch 8/544, Batch Accuracy: 87.50%, Loss: 0.0002
Epoch 1, Batch 9/544, Batch Accuracy: 88.89%, Loss: 0.0004
Epoch 1, Batch 10/544, Batch Accuracy: 90.00%, Loss: 0.0001
Epoch 1, Batch 11/544, Batch Accuracy: 90.91%, Loss: 0.0001
Epoch 1, Batch 12/544, Batch Accuracy: 91.67%, Loss: 0.0000
Epoch 1, Batch 13/544, Batch Accuracy: 92.31%, Loss: 0.0001
Epoch 1, Batch 14/544, Batch Accuracy: 92.86%, Loss: 0.0000
Epoch 1, Batch 15/544, Batch Accuracy: 93.33%, Loss: 0.0000
Epoch 1, Batch 16/544, Batch Accuracy: 93.75%, Loss: 0.0000


# Preparing for inference
### I ASSUMED THE INPUT FILE IS CALLED 'test.ll' you should change its name or directory

In [ ]:
llfilePath = 'test.ll'



# Load encoder
#Load the encoder for each token
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)
padderLabel = encoder.transform(['<pad>'])[0]
print(f"padderLabel is {padderLabel}")


#Load model
with open('LSTM.pkl', 'rb') as f:
    model = pickle.load(f)
model.eval()

classificationTest = dict()
Embedding = dict()

# if input file is llvm file

llTokenized_heads, llTokenized_bodies = CodePreprocessing.functions_preprocessing(llfilePath, '', writeJson=False)
for i in range(len(llTokenized_heads)):

    # Pad to max size
    Test_arr = np.zeros(max_size, dtype='int')
    Test_arr[0: len(llTokenized_bodies[i])] = llTokenized_bodies[i]
    Test_arr[len(llTokenized_bodies[i]) : max_size] = padderLabel

    torch.cuda.empty_cache()
    new_X_tensor = torch.LongTensor(Test_arr).to(device='cuda')

    
    with torch.no_grad():
        model.eval()
        input_tensor = torch.LongTensor(new_X_tensor).unsqueeze(0)  # Add a batch dimension
        output, sentence_embedding = model(input_tensor)
        predicted_class = torch.argmax(output, dim=1).item()

    print(f'Predicted class: {predicted_class}')
    print(f'Sentence embedding: {sentence_embedding}')
    sentence_embedding = sentence_embedding.detach().numpy()
    print(f'Sentence embedding: {sentence_embedding}')

    classificationTest[llTokenized_heads[i]] = predicted_class
    Embedding[llTokenized_heads[i]] = sentence_embedding

In [ ]:
'''
# Define hyperparameters and load your model
# ...

# Preprocess your input sentence
# ...

# Perform inference and extract the sentence embedding
with torch.no_grad():
    model.eval()
    input_tensor = torch.LongTensor(preprocessed_sentence).unsqueeze(0)  # Add a batch dimension
    output, sentence_embedding = model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

print(f'Predicted class: {predicted_class}')
print(f'Sentence embedding: {sentence_embedding}')
sentence_embedding = sentence_embedding.detach().numpy()
print(f'Sentence embedding: {sentence_embedding}')
'''